In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Disables warning messages for beauty
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Importing all files as dataframes
code_df = pd.read_csv('data/codes.csv', delimiter=';')
transactions_df=pd.read_csv('data/transactions.csv', sep=';')
train_df=pd.read_csv('data/train_set.csv', sep=';', encoding = "ISO-8859-1")
test_df=pd.read_csv('data/test_set.csv', encoding = "ISO-8859-1")

code_df.head()

,code,code_description
0,5944,"Магазины по продаже часов, ювелирных изделий и..."
1,5621,Готовые сумочные изделия
2,5697,"Услуги по переделке, починке и пошиву одежды"
3,7995,Транзакции по азартным играм
4,5137,"Мужская, женская и детская спец-одежда"


In [4]:
transactions_df

,client_id,datetime,code,type,sum
0,96372458,421 06:33:15,6011,2010,-561478.94
1,24567813,377 17:20:40,6011,7010,67377.47
2,21717441,55 13:38:47,6011,2010,-44918.32
3,14331004,263 12:57:08,6011,2010,-3368873.66
4,85302434,151 10:34:12,4814,1030,-3368.87
...,...,...,...,...,...
130034,15836839,147 11:50:53,5411,1010,-26344.59
130035,28369355,305 11:59:34,4829,2330,-24705.07
130036,40949707,398 21:13:58,5411,1110,-40353.72
130037,7174462,409 13:58:14,5411,1010,-25536.06


In [5]:
X_train=transactions_df[transactions_df.client_id.isin(train_df.client_id)]
X_test=transactions_df[transactions_df.client_id.isin(test_df.client_id)]